In [1]:
from pyspark.sql import SparkSession

spark = (SparkSession
        .builder
        .appName("PythonMnMCount")
        .getOrCreate())

In [2]:
#EJERCICIO 1 : Leerlo tratando de que Spark infiera el tipo de dato de cada columna, y cachearlo

In [3]:
# LÍNEA EVALUABLE, NO RENOMBRAR LAS VARIABLES
accidentesDF = (spark.read.format("csv")
        .option("header", "true")
        .option("inferSchema", "true")
        .load('C:/Users/laura.serrano/Desktop/accidents_uk.csv')).cache() 


In [4]:
from pyspark.sql.types import DoubleType
assert(accidentesDF.schema[1].dataType == DoubleType())

In [5]:
accidentesDF.printSchema()

root
 |-- Accident_Index: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Region: string (nullable = true)
 |-- Urban_or_Rural_Area: string (nullable = true)
 |-- X1st_Road_Class: string (nullable = true)
 |-- Driver_IMD_Decile: integer (nullable = true)
 |-- Speed_limit: integer (nullable = true)
 |-- Road_Type: string (nullable = true)
 |-- Road_Surface_Conditions: string (nullable = true)
 |-- Weather: string (nullable = true)
 |-- High_Wind: string (nullable = true)
 |-- Lights: string (nullable = true)
 |-- Datetime: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Season: integer (nullable = true)
 |-- Month_of_Year: integer (nullable = true)
 |-- Day_of_Month: integer (nullable = true)
 |-- Day_of_Week: integer (nullable = true)
 |-- Hour_of_Day: double (nullable = true)
 |-- Number_of_Vehicles: integer (nullable = true)
 |-- Age_of_Driver: integer (nullable = true)
 |-- Age_of_Vehicle: integer (nul

In [6]:
#EJERCICIO 2: Discretizar la variable Age_of_Vehicle utilizando un bucketizer (sin crear un pipeline) en los puntos de corte (0, 2, 5, 10, 15, 20, 35). La discretización debe quedar en una nueva columna de tipo Double llamada Edad_Vehiculo.

In [7]:
from pyspark.ml.feature import Bucketizer
# LÍNEAS EVALUABLES, NO RENOMBRAR LAS VARIABLES
bucketizer = Bucketizer(splits=[0.0, 2.0, 5.0, 10.0, 15.0, 20.0, 35.0], inputCol="Age_of_Vehicle", outputCol="Edad_Vehiculo")
accidentesBucketizedDF = bucketizer.transform(accidentesDF)
accidentesBucketizedDF.select("Edad_vehiculo").show()


+-------------+
|Edad_vehiculo|
+-------------+
|          2.0|
|          1.0|
|          2.0|
|          1.0|
|          3.0|
|          1.0|
|          3.0|
|          2.0|
|          0.0|
|          1.0|
|          2.0|
|          1.0|
|          1.0|
|          3.0|
|          2.0|
|          3.0|
|          2.0|
|          1.0|
|          2.0|
|          2.0|
+-------------+
only showing top 20 rows



In [8]:
assert("Edad_Vehiculo" in accidentesBucketizedDF.columns)
assert(accidentesBucketizedDF.schema.fields)

In [9]:
#EJERCICIO 3: Crear un nuevo DF donde la columna "Age_of_Driver" ha sido reemplazada por otra de tipo string en la que los valores 1 y 2 son "Adolescente", los valores 3 y 4 por "Joven", los valores 5 y 6 por "Adulto", y los demás valores se dejan sin modificar.

In [10]:
# LÍNEA EVALUABLE, NO RENOMBRAR VARIABLES
from pyspark.sql import functions as F
accidentesAgeDF = accidentesDF.withColumn("Age_of_Driver", F.when(((F.col("Age_of_Driver") == 1) | (F.col("Age_of_Driver") == 2)), "Adolescente")\
                                                            .when(((F.col("Age_of_Driver") == 3) | (F.col("Age_of_Driver") == 4)), "Joven")\
                                                            .when(((F.col("Age_of_Driver") == 5) | (F.col("Age_of_Driver") == 6)), "Adulto")\
                                                            .otherwise(F.col("Age_of_Driver")))



accidentesAgeDF.printSchema() # the column is still in the same position but has now string type

root
 |-- Accident_Index: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Region: string (nullable = true)
 |-- Urban_or_Rural_Area: string (nullable = true)
 |-- X1st_Road_Class: string (nullable = true)
 |-- Driver_IMD_Decile: integer (nullable = true)
 |-- Speed_limit: integer (nullable = true)
 |-- Road_Type: string (nullable = true)
 |-- Road_Surface_Conditions: string (nullable = true)
 |-- Weather: string (nullable = true)
 |-- High_Wind: string (nullable = true)
 |-- Lights: string (nullable = true)
 |-- Datetime: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Season: integer (nullable = true)
 |-- Month_of_Year: integer (nullable = true)
 |-- Day_of_Month: integer (nullable = true)
 |-- Day_of_Week: integer (nullable = true)
 |-- Hour_of_Day: double (nullable = true)
 |-- Number_of_Vehicles: integer (nullable = true)
 |-- Age_of_Driver: string (nullable = true)
 |-- Age_of_Vehicle: integer (null

In [11]:
assert(dict(accidentesAgeDF.dtypes)["Age_of_Driver"] == "string")
collectedDF = accidentesAgeDF.groupBy("Age_of_Driver").count().orderBy("count").collect()
print(collectedDF)
assert((collectedDF[0]["count"] == 9195) & (collectedDF[0]["Age_of_Driver"] == "8"))
assert((collectedDF[1]["count"] == 13338) & (collectedDF[1]["Age_of_Driver"] == "7"))
assert((collectedDF[2]["count"] == 57174) & (collectedDF[2]["Age_of_Driver"] == "Adolescente"))
assert((collectedDF[3]["count"] == 67138) & (collectedDF[3]["Age_of_Driver"] == "Adulto"))
assert((collectedDF[4]["count"] == 104987) & (collectedDF[4]["Age_of_Driver"] == "Joven"))

[Row(Age_of_Driver='8', count=9195), Row(Age_of_Driver='7', count=13338), Row(Age_of_Driver='Adolescente', count=57174), Row(Age_of_Driver='Adulto', count=67138), Row(Age_of_Driver='Joven', count=104987)]


In [12]:
#EJERCICIO 4:Partiendo de accidentesDF, crear un nuevo DataFrame de una sola fila que contenga, por este orden de columnas, el número de categorías existentes para el propósito del viaje, para el tipo de maniobra del vehículo, para las condiciones de la calzada y para la severidad del accidente. Pista: crear las columnas al vuelo con select(). Renombrar cada columna de conteo para que se llame igual que la propia columna que estamos contando.

In [14]:
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import col
# LÍNEA EVALUABLE, NO RENOMBRAR VARIABLES
numeroCategoriasDF = (accidentesDF.select("Driver_Journey_Purpose", "Vehicle_Manoeuvre", "Road_Surface_Conditions", "Accident_Severity")\
                .agg(countDistinct(col("Driver_Journey_Purpose")).alias("num_pourpose"),\
     countDistinct(col("Vehicle_Manoeuvre")).alias("num_maniobras"),\
     countDistinct(col("Road_Surface_Conditions")).alias("num_condicion_calzada"),\
     countDistinct(col("Accident_Severity")).alias("num_severidad")))
numeroCategoriasDF.show()



+------------+-------------+---------------------+-------------+
|num_pourpose|num_maniobras|num_condicion_calzada|num_severidad|
+------------+-------------+---------------------+-------------+
|           5|           11|                    5|            2|
+------------+-------------+---------------------+-------------+



In [15]:
assert(len(numeroCategoriasDF.columns) == 4)
assert(numeroCategoriasDF.count() == 1)

In [16]:
#EJERCICIO 5: Partiendo de accidentesAgeDF definido anteriormente, crear un nuevo DataFrame con tantas filas como posibles 
#propósitos de un viaje, y tantas columnas como rangos de edad habíamos distinguido en dicho DataFrame más una 
#(la del propósito del viaje). Las columnas deben llamarse igual que cada uno de los niveles posibles de rangos de edad. 
#Cada casilla del nuevo DataFrame deberá contener el porcentaje del número de accidentes ocurridos en ese tipo de viaje (fila) 
#para ese rango de edad (columna), medido sobre el total de accidentes ocurridos para ese tipo de viaje.

In [17]:
from pyspark.sql.functions import lit
from pyspark.sql import functions as F
from pyspark.sql.functions import count
viajesPorEdadDF = (accidentesAgeDF.groupBy("Driver_Journey_Purpose")\
                   .pivot("Age_of_Driver")\
                   .count()\
                   .withColumn("Total", F.col("Adolescente")+F.col("Adulto")+F.col("Joven")+F.col("7")+F.col("8"))\
                   .withColumn("Adolescente", F.col("Adolescente") /F.col("Total"))\
                   .withColumn("Joven", (F.col("Joven") / F.col("Total")))\
                   .withColumn("Adulto",( F.col("Adulto")/F.col("Total")))\
                   .withColumn("7", (F.col("7") / F.col("Total")))\
                   .withColumn("8", (F.col("8") / F.col("Total"))))\
                   .drop("Total")

accidentesAgeDF.groupBy("Driver_Journey_Purpose")\
                   .pivot("Age_of_Driver")\
                   .count()\
.select(F.sum(col("8"))).show()


+------+
|sum(8)|
+------+
|  9195|
+------+



In [18]:
viajesPorEdadDF.printSchema()

root
 |-- Driver_Journey_Purpose: string (nullable = true)
 |-- 7: double (nullable = true)
 |-- 8: double (nullable = true)
 |-- Adolescente: double (nullable = true)
 |-- Adulto: double (nullable = true)
 |-- Joven: double (nullable = true)



In [19]:
viajesPorEdadDF.orderBy("Driver_Journey_Purpose").collect()[0]

Row(Driver_Journey_Purpose='Commuting to/from work', 7=0.012527948326649396, 8=0.0025197856407708414, Adolescente=0.236327501153423, Adulto=0.2791993469851297, Joven=0.46942541789402703)

In [20]:
assert(len(viajesPorEdadDF.columns) >= 6)
assert("7" in viajesPorEdadDF.columns)
assert("8" in viajesPorEdadDF.columns)
assert("Adolescente" in viajesPorEdadDF.columns)
assert("Joven" in viajesPorEdadDF.columns)
assert("Adulto" in viajesPorEdadDF.columns)
assert(viajesPorEdadDF.columns[0] == "Driver_Journey_Purpose")
assert(viajesPorEdadDF.count() == 5)
commuting = viajesPorEdadDF.orderBy("Driver_Journey_Purpose").collect()[0]
assert(commuting.Driver_Journey_Purpose.startswith("Commuting"))
assert(abs(commuting['7'] - 0.012527948326649396) < 0.001)
assert(abs(commuting['8'] - 0.002519785640770) < 0.001)
assert(abs(commuting.Adolescente - 0.236327501153423) < 0.001)
assert(abs(commuting.Adulto - 0.2791993469851297) < 0.001)
assert(abs(commuting.Joven - 0.46942541789402703) < 0.001)